# Subway Dataset 2019 Analysis


## Purpose of this Notebook:
- This notebook is my data pipeline to clean and analyze the MTA Subway Dataset for 2019.
- In a seperate notebook I will build, train and evaluate prediction models to capture the relationship between busyness and other features such as day of the week, time of day, location and weather.
- The reason for choosing 2019 data is that it is the most recent avaible dataset that is not skewed by Covid-19.
- The cleaning process will provide a dataset which can help predict busyness at certain times of the day and week for various locations in Manhattan.

### Main Goals of Data Cleaning:
- Will need to determine all the turnstile differences for entries/exits between current and last value for each station.
- This can give me a representation of how many people enter/exit station X in a given time period
- Do this for only stations in Manhattan as that is the scope of this project

In [1]:
#Imports for this notebook
import pandas as pd 

In [2]:
##Writing installed packages to requirements.txt file
!pip freeze > requirements.txt

### Overview of the data:
- The dataset provided by the MTA is not that useful in its raw form. It required external datasets to link to it in order to get more functionality from the turnstile data.
- This blog post was of great help in understanding and processing the 2019 turnstile data.https://medium.com/qri-io/taming-the-mtas-unruly-turnstile-data-c945f5f96ba0
#### Turnstile Data:
- turnstile_data is the dataset obtained from: https://data.ny.gov/Transportation/Turnstile-Usage-Data-2019/xfn5-qji9
- This dataset contains all the info in 2019 for all the turnstiles in the MTA system
#### Station Data:
- station_data is a dataset obtained from: http://web.mta.info/developers/data/nyct/subway/Stations.csv
- This dataset contains information for all the stations in the MTA system
#### How to link these:
- I need a lookup table to map each station in the turnstile data to the exact co-ordinates of each station. This is necessary as the point in processing this data is to provide a busyness prediction for location. This will vary from station to station. It will also be helpful in displaying a heatmap on our web app.
- lookup_data provides a link between the two in the form of a column named remote or unit id that links each remote unit id to each station

#### Reading in Turnstile Data:

In [3]:
turnstile_data = pd.read_csv('Turnstile_Usage_Data__2019.csv')
turnstile_data.head()

,C/A,Unit,SCP,Station,Line Name,Division,Date,Time,Description,Entries,Exits
0,A033,R170,02-00-05,14 ST-UNION SQ,LNQR456W,BMT,12/27/2019,00:00:00,REGULAR,17538854,7031168
1,A033,R170,02-00-02,14 ST-UNION SQ,LNQR456W,BMT,12/27/2019,00:00:00,REGULAR,14983900,14554087
2,A033,R170,02-06-00,14 ST-UNION SQ,LNQR456W,BMT,12/27/2019,00:00:00,REGULAR,769115,559221
3,A033,R170,02-00-03,14 ST-UNION SQ,LNQR456W,BMT,12/27/2019,00:00:00,REGULAR,7191422,8417203
4,A033,R170,02-06-01,14 ST-UNION SQ,LNQR456W,BMT,12/27/2019,00:00:00,REGULAR,71047673,20925389


#### Reading in Station Data:

In [4]:
with open('stations.csv') as file:
    station_data = pd.read_csv(file)
station_data.head()

,ogc_fid,station id,complex id,gtfs stop id,division,line,stop name,borough,daytime routes,structure,gtfs latitude,gtfs longitude,north direction label,south direction label
0,1,1.0,1,R01,BMT,Astoria,Astoria - Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034,NaN,Manhattan
1,2,2.0,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843,Ditmars Blvd,Manhattan
2,3,3.0,3,R04,BMT,Astoria,30 Av,Q,N W,Elevated,40.766779,-73.921479,Astoria - Ditmars Blvd,Manhattan
3,4,4.0,4,R05,BMT,Astoria,Broadway,Q,N W,Elevated,40.761820,-73.925508,Astoria - Ditmars Blvd,Manhattan
4,5,5.0,5,R06,BMT,Astoria,36 Av,Q,N W,Elevated,40.756804,-73.929575,Astoria - Ditmars Blvd,Manhattan


#### Reading in Lookup Data:

In [5]:
with open('remote_complex_lookup.csv') as file:
    lookup_data = pd.read_csv(file)
lookup_data.head()

,remote,booth,complex_id,station,line_name,division
0,R001,A060,635.0,WHITEHALL ST,R1,BMT
1,R001,A058,635.0,WHITEHALL ST,R1,BMT
2,R001,R101S,635.0,SOUTH FERRY,R1,IRT
3,R002,A077,628.0,FULTON ST,ACJZ2345,BMT
4,R002,A081,628.0,FULTON ST,ACJZ2345,BMT


In [6]:
#count the number of null values for complex id
lookup_data['complex_id'].isnull().sum()

36

In [7]:
#change the complex id column to object instead of float
lookup_data['complex_id'] = lookup_data['complex_id'].astype(str)
lookup_data.head()

,remote,booth,complex_id,station,line_name,division
0,R001,A060,635.0,WHITEHALL ST,R1,BMT
1,R001,A058,635.0,WHITEHALL ST,R1,BMT
2,R001,R101S,635.0,SOUTH FERRY,R1,IRT
3,R002,A077,628.0,FULTON ST,ACJZ2345,BMT
4,R002,A081,628.0,FULTON ST,ACJZ2345,BMT


### First lets look at all the station_data dataset
- First I will drop all rows that are not in Manhattan

In [8]:
for i in range(len(station_data)):
    if station_data['borough'][i] == 'Q' or  station_data['borough'][i] == 'Bk' or station_data['borough'][i] == 'Bx'or station_data['borough'][i] == 'SI':
        station_data.drop(i, inplace=True)

station_data.head()

,ogc_fid,station id,complex id,gtfs stop id,division,line,stop name,borough,daytime routes,structure,gtfs latitude,gtfs longitude,north direction label,south direction label
6,7,7.0,613,R11,BMT,Astoria,Lexington Av/59 St,M,N W R,Subway,40.762660,-73.967258,Queens,Downtown & Brooklyn
7,8,8.0,8,R13,BMT,Astoria,5 Av/59 St,M,N W R,Subway,40.764811,-73.973347,Queens,Downtown & Brooklyn
8,9,9.0,9,R14,BMT,Broadway - Brighton,57 St - 7 Av,M,N Q R W,Subway,40.764664,-73.980658,Uptown & Queens,Downtown & Brooklyn
9,10,10.0,10,R15,BMT,Broadway - Brighton,49 St,M,N R W,Subway,40.759901,-73.984139,Uptown & Queens,Downtown & Brooklyn
10,11,11.0,611,R16,BMT,Broadway - Brighton,Times Sq - 42 St,M,N Q R W,Subway,40.754672,-73.986754,Uptown & Queens,Downtown & Brooklyn


In [9]:
station_data.tail(20)

,ogc_fid,station id,complex id,gtfs stop id,division,line,stop name,borough,daytime routes,structure,gtfs latitude,gtfs longitude,north direction label,south direction label
472,473,475.0,475,Q05,IND,Second Av,96 St,M,Q,Subway,40.784318,-73.947152,NaN,Downtown & Brooklyn
473,474,476.0,476,Q04,IND,Second Av,86 St,M,Q,Subway,40.777891,-73.951787,Uptown,Downtown & Brooklyn
474,475,477.0,477,Q03,IND,Second Av,72 St,M,Q,Subway,40.768799,-73.958424,Uptown,Downtown & Brooklyn
496,497,NaN,R468,NaN,RIT,NaN,Roosevelt Island Tram - Eastbound,NaN,NaN,NaN,40.761186,-73.964191,NaN,NaN
497,498,NaN,R469,NaN,RIT,NaN,Roosevelt Island Tram - Westbound,NaN,NaN,NaN,40.757307,-73.954097,NaN,NaN
498,499,NaN,R540,NaN,PTH,NaN,World Trade Center,NaN,NaN,NaN,40.712052,-74.014129,NaN,NaN
499,500,NaN,R541,NaN,PTH,NaN,33 St,NaN,NaN,NaN,40.747850,-73.989875,NaN,NaN
500,501,NaN,R542,NaN,PTH,NaN,23 St,NaN,NaN,NaN,40.742672,-73.994177,NaN,NaN
501,502,NaN,R543,NaN,PTH,NaN,Exchange Pl,NaN,NaN,NaN,40.716554,-74.033372,NaN,NaN
502,503,NaN,R544,NaN,PTH,NaN,Harrison,NaN,NaN,NaN,40.740472,-74.157344,NaN,NaN


In [10]:
#take out the last 17 station names and put them in a list
station_names = station_data['stop name'].tail(17).tolist()
station_names

['Roosevelt Island Tram - Eastbound',
 'Roosevelt Island Tram - Westbound',
 'World Trade Center',
 '33 St',
 '23 St',
 'Exchange Pl',
 'Harrison',
 '14 St',
 'Pavonia/Newport',
 '9 St',
 'Christopher St',
 'Newark Penn Station',
 'Hoboken',
 'Grove St',
 'Journal Sq',
 'Howard Beach',
 'Jamaica']

In [11]:
station_names_to_drop=['Roosevelt Island Tram - Eastbound',
 'Roosevelt Island Tram - Westbound',
 'Exchange Pl',
 'Harrison',
 'Pavonia/Newport',
 '9 St',
 'Newark Penn Station',
 'Hoboken',
 'Grove St',
 'Journal Sq',
 'Howard Beach',
 'Jamaica']

 # Create a boolean mask to identify rows with names are in the list
mask = station_data['stop name'].isin(station_names_to_drop)

# Apply the mask to drop the rows
station_data = station_data[~mask]

station_data.tail(10)
    

,ogc_fid,station id,complex id,gtfs stop id,division,line,stop name,borough,daytime routes,structure,gtfs latitude,gtfs longitude,north direction label,south direction label
470,471,469.0,610,901,IRT,Lexington - Shuttle,Grand Central - 42 St,M,S,Subway,40.752769,-73.979189,Times Sq,NaN
471,472,471.0,471,726,IRT,Flushing,34 St - 11 Av,M,7,Subway,40.755882,-74.001910,Queens,NaN
472,473,475.0,475,Q05,IND,Second Av,96 St,M,Q,Subway,40.784318,-73.947152,NaN,Downtown & Brooklyn
473,474,476.0,476,Q04,IND,Second Av,86 St,M,Q,Subway,40.777891,-73.951787,Uptown,Downtown & Brooklyn
474,475,477.0,477,Q03,IND,Second Av,72 St,M,Q,Subway,40.768799,-73.958424,Uptown,Downtown & Brooklyn
498,499,NaN,R540,NaN,PTH,NaN,World Trade Center,NaN,NaN,NaN,40.712052,-74.014129,NaN,NaN
499,500,NaN,R541,NaN,PTH,NaN,33 St,NaN,NaN,NaN,40.747850,-73.989875,NaN,NaN
500,501,NaN,R542,NaN,PTH,NaN,23 St,NaN,NaN,NaN,40.742672,-73.994177,NaN,NaN
503,504,NaN,R545,NaN,PTH,NaN,14 St,NaN,NaN,NaN,40.738507,-73.998741,NaN,NaN
506,507,NaN,R548,NaN,PTH,NaN,Christopher St,NaN,NaN,NaN,40.732593,-74.008924,NaN,NaN


In [12]:
#set the last 5 rows column[gtfs stop id] to the complex id value
station_data.loc[station_data.tail(5).index, 'gtfs stop id'] = station_data['complex id'].tail(5)

In [13]:
#change complex id in station to have .0 and be of type string
station_data['complex id'] = station_data['complex id'].astype(str)
station_data['complex id'] = station_data['complex id'] + '.0'

### Now match up complex_id in station_data with complex_id in lookup_data

In [14]:
station_remote_info = pd.merge(station_data, lookup_data, left_on='complex id', right_on='complex_id')
station_remote_info.head()

,ogc_fid,station id,complex id,gtfs stop id,division_x,line,stop name,borough,daytime routes,structure,gtfs latitude,gtfs longitude,north direction label,south direction label,remote,booth,complex_id,station,line_name,division_y
0,7,7.0,613.0,R11,BMT,Astoria,Lexington Av/59 St,M,N W R,Subway,40.76266,-73.967258,Queens,Downtown & Brooklyn,R016,N305A,613.0,LEXINGTON-53 ST,EM6,IND
1,7,7.0,613.0,R11,BMT,Astoria,Lexington Av/59 St,M,N W R,Subway,40.76266,-73.967258,Queens,Downtown & Brooklyn,R017,N306,613.0,LEXINGTON-53 ST,EM6,IND
2,7,7.0,613.0,R11,BMT,Astoria,Lexington Av/59 St,M,N W R,Subway,40.76266,-73.967258,Queens,Downtown & Brooklyn,R017,N305,613.0,LEXINGTON-53 ST,EM6,IND
3,7,7.0,613.0,R11,BMT,Astoria,Lexington Av/59 St,M,N W R,Subway,40.76266,-73.967258,Queens,Downtown & Brooklyn,R050,R244,613.0,59 ST,456NQR,IRT
4,7,7.0,613.0,R11,BMT,Astoria,Lexington Av/59 St,M,N W R,Subway,40.76266,-73.967258,Queens,Downtown & Brooklyn,R050,R244A,613.0,59 ST,456NQR,IRT


In [15]:
#drop the following columns: ogc_fid, gtfs stop id, division_x, line, daytime routes, structure, north direction label, south direction label, line_name, division_y
station_remote_info.drop(['ogc_fid', 'station id', 'complex id','gtfs stop id', 'division_x', 'line', 'borough','daytime routes', 'structure', 'north direction label', 'south direction label', 'line_name', 'division_y', 'complex_id','booth'], axis=1, inplace=True)

station_remote_info.head()

,stop name,gtfs latitude,gtfs longitude,remote,station
0,Lexington Av/59 St,40.76266,-73.967258,R016,LEXINGTON-53 ST
1,Lexington Av/59 St,40.76266,-73.967258,R017,LEXINGTON-53 ST
2,Lexington Av/59 St,40.76266,-73.967258,R017,LEXINGTON-53 ST
3,Lexington Av/59 St,40.76266,-73.967258,R050,59 ST
4,Lexington Av/59 St,40.76266,-73.967258,R050,59 ST


In [16]:
#rename the columns:
station_remote_info.rename(columns={'remote ':'remote_unit_id','gtfs longitude':'lon','gtfs latitude': 'lat', 'stop name': 'station_name', 'station':'short_name'}, inplace=True)
station_remote_info.head()


,station_name,lat,lon,remote_unit_id,short_name
0,Lexington Av/59 St,40.76266,-73.967258,R016,LEXINGTON-53 ST
1,Lexington Av/59 St,40.76266,-73.967258,R017,LEXINGTON-53 ST
2,Lexington Av/59 St,40.76266,-73.967258,R017,LEXINGTON-53 ST
3,Lexington Av/59 St,40.76266,-73.967258,R050,59 ST
4,Lexington Av/59 St,40.76266,-73.967258,R050,59 ST


In [17]:
#remove duplicate rows
station_remote_info.drop_duplicates(subset=['remote_unit_id'], inplace=True)
#reset index
station_remote_info.reset_index(drop=True, inplace=True)

In [18]:
len(station_remote_info)

143

### Result:
- This leaves us with a dataset that contains all the stations in Manhattan, their co-ordinates and their remote unit id to link to the turnstile data

### Lets now look at the turnstile_data dataset
- First I should drop all the rows that concern Manhattan stations only

In [19]:
len(turnstile_data)

10467101

In [20]:
#reduce the number of rows in turnstile data by only including matches for the following condition: 
#remote in remote=unit. This is a unique id for each subsection of a station in Manhattan
turnstile_data = turnstile_data[turnstile_data['Unit'].isin(station_remote_info['remote_unit_id'])]
len(turnstile_data)

4617524

### This should now be all the turnstile data for manhattan stations in 2019

- Use the remote_unit_id for linking a station to a turnstile
- station_remote_info is useful as it has the station name, lat and lon and also a link to the turnstile data for that station
- Can now proceed with processing the turnstile data

- I will convert date and time to unix timestamp, name: timestamp
- I will combine C/A, Unit, SCP and the timestamp to create a unique identifier for each row, name: id
- I will combine C/A, Unit and SCP to create a unique id for each, name: turnstile_id
- I will calculate the entry and exit values for each row, name: entry_diff and exit_diff

In [21]:
#create a new dateframe name for turnstile data
turnstile_data_man = turnstile_data.copy()

In [22]:
turnstile_data_man.head()

,C/A,Unit,SCP,Station,Line Name,Division,Date,Time,Description,Entries,Exits
0,A033,R170,02-00-05,14 ST-UNION SQ,LNQR456W,BMT,12/27/2019,00:00:00,REGULAR,17538854,7031168
1,A033,R170,02-00-02,14 ST-UNION SQ,LNQR456W,BMT,12/27/2019,00:00:00,REGULAR,14983900,14554087
2,A033,R170,02-06-00,14 ST-UNION SQ,LNQR456W,BMT,12/27/2019,00:00:00,REGULAR,769115,559221
3,A033,R170,02-00-03,14 ST-UNION SQ,LNQR456W,BMT,12/27/2019,00:00:00,REGULAR,7191422,8417203
4,A033,R170,02-06-01,14 ST-UNION SQ,LNQR456W,BMT,12/27/2019,00:00:00,REGULAR,71047673,20925389


In [23]:
#convert date and time to unix timestamp
turnstile_data_man['DateTime'] = pd.to_datetime(turnstile_data_man['Date'] + ' ' + turnstile_data_man['Time'])

turnstile_data_man['Timestamp'] = turnstile_data_man['DateTime'].apply(lambda x: x.timestamp())

In [24]:
#combine c/a, unit, and scp to create a unique id for each turnstile
turnstile_data_man['turnstile_id'] = turnstile_data_man['C/A'] +' '+ turnstile_data_man['Unit'] +' '+  turnstile_data_man['SCP']

In [25]:
#combine c/a, unit, scp, and timestamp to create a unique id for each turnstile at a given time
turnstile_data_man['id'] = turnstile_data_man['turnstile_id']+ ' '+  turnstile_data_man['Timestamp'].astype(str)

In [26]:
#can now drop c/a, scp, line name, divison,date, time, datetime
turnstile_data_man.drop(['C/A', 'SCP', 'Line Name', 'Division', 'Date', 'Time', 'DateTime'], axis=1, inplace=True)

In [27]:
turnstile_data_man.columns

Index(['Unit', 'Station', 'Description', 'Entries',
       'Exits                                                     ',
       'Timestamp', 'turnstile_id', 'id'],
      dtype='object')

In [28]:
#re-orient the columns
#1st will be id, 2nd turnstile id, then unit, timestamp, station, entires,exits and description
new_order = ['id', 'turnstile_id', 'Unit', 'Timestamp', 'Station', 'Entries', 'Exits                                                     ', 'Description']
turnstile_data_man = turnstile_data_man[new_order]


In [29]:
#re-name the columns:
turnstile_data_man.rename(columns={'Unit':'remote_unit_id', 'Timestamp':'time_stamp','Station':'station_name', 'Entries':'entries', 'Exits                                                     ':'exits', 'Description':'description'}, inplace=True)
len(turnstile_data_man)

4617524

In [30]:
#check for duplicate ids, Nan and other possible error values
#count duplicates:
# is it possible that entry and exit values differ for the duplicate id?
#create a new column
turnstile_data_man['check_dup']=turnstile_data_man['id']+' '+turnstile_data_man['entries'].astype(str)+' '+turnstile_data_man['exits'].astype(str)

duplicate_count = turnstile_data_man['id'].duplicated().sum()
print("Number of duplicates in 'id' column:", duplicate_count)
dup_count = turnstile_data_man['check_dup'].duplicated().sum()
print("Number of duplicates in check_dup column:", dup_count)
nan_count=turnstile_data_man.isnull().sum()
print("Number of null values in all columns:\n", nan_count)
# count the number of times "RECOVR AUD" appears in the description column
r_aud=turnstile_data_man['description'].str.contains('RECOVR AUD').sum()
print("Number of times 'RECOVR AUD' appears in the description column:", r_aud)


Number of duplicates in 'id' column: 181348
Number of duplicates in check_dup column: 181211
Number of null values in all columns:
 id                0
turnstile_id      0
remote_unit_id    0
time_stamp        0
station_name      0
entries           0
exits             0
description       0
check_dup         0
dtype: int64
Number of times 'RECOVR AUD' appears in the description column: 21902


In [31]:
#drop column check_dup
turnstile_data_man.drop(['check_dup'], axis=1, inplace=True)
#drop all the duplicate id values as only 137 of them have diff entry and exit values
turnstile_data_man = turnstile_data_man.drop_duplicates(subset=['id'])

In [32]:
#sort by time stamp and name of station, drop all data before 1546300800
turnstile_data_man.sort_values(by=['time_stamp', 'station_name'], inplace=True)

#drop all data before 1546300800
turnstile_data_man = turnstile_data_man[turnstile_data_man['time_stamp'] >= 1546290000]

#### Left to do:
- Calculate entry and exit differences for each turnstile
- Drop the description column

In [33]:
#need to reset the index:
turnstile_data_man.reset_index(drop=True, inplace=True)
#create a copy with the new entries and exits
df_turn=turnstile_data_man.copy()
#function to get the difference between the current and previous entry and exit values
def calc_entry_exit_diff(group):
    group['entries'] = group['entries'].diff()
    group['exits'] = group['exits'].diff()
    return group
#apply the function to the dataframe
df_turn[['entries', 'exits']] = df_turn.groupby('turnstile_id',group_keys=False)[['entries', 'exits']].apply(calc_entry_exit_diff)
#fill nan values with 0
df_turn['entries'] = df_turn['entries'].fillna(0)
df_turn['exits'] = df_turn['exits'].fillna(0)

df_turn.head()

,id,turnstile_id,remote_unit_id,time_stamp,station_name,entries,exits,description
0,R138 R293 00-03-03 1546290000.0,R138 R293 00-03-03,R293,1.546290e+09,34 ST-PENN STA,0.0,0.0,REGULAR
1,R138 R293 00-02-05 1546290000.0,R138 R293 00-02-05,R293,1.546290e+09,34 ST-PENN STA,0.0,0.0,REGULAR
2,R138 R293 00-02-03 1546290000.0,R138 R293 00-02-03,R293,1.546290e+09,34 ST-PENN STA,0.0,0.0,REGULAR
3,R138 R293 00-00-02 1546290000.0,R138 R293 00-00-02,R293,1.546290e+09,34 ST-PENN STA,0.0,0.0,REGULAR
4,R138 R293 00-03-01 1546290000.0,R138 R293 00-03-01,R293,1.546290e+09,34 ST-PENN STA,0.0,0.0,REGULAR


In [34]:
len(df_turn)

4399477

In [35]:
#reset the index
df_turn.reset_index(drop=True, inplace=True)

#create boolean masks for filtering
entries_mask = (df_turn['entries'] <= 2800) & (df_turn['entries'] >= 5)
exits_mask = (df_turn['exits'] <= 2800) & (df_turn['exits'] >= 5)

#apply the masks to filter the DataFrame
df_turn = df_turn[entries_mask & exits_mask]

len(df_turn)


3390842

In [36]:
#drop description and id
df_turn.drop(['id', 'description'], axis=1, inplace=True)

In [37]:
#change entries, exits and the difference between them to integers
df_turn['entries'] = df_turn['entries'].astype(int)
df_turn['exits'] = df_turn['exits'].astype(int)
#change time stamp to int
df_turn['time_stamp'] = df_turn['time_stamp'].astype(int)
#drop the r in remtote unit id and convert to int
df_turn['remote_unit_id'] = df_turn['remote_unit_id'].str.replace('R', '')
df_turn['remote_unit_id'] = df_turn['remote_unit_id'].astype(int)

In [38]:
df_turn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3390842 entries, 2105 to 4399476
Data columns (total 6 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   turnstile_id    object
 1   remote_unit_id  int64 
 2   time_stamp      int64 
 3   station_name    object
 4   entries         int64 
 5   exits           int64 
dtypes: int64(4), object(2)
memory usage: 181.1+ MB


In [39]:
df_turn.head()


,turnstile_id,remote_unit_id,time_stamp,station_name,entries,exits
2105,R138 R293 00-02-03,293,1546304400,34 ST-PENN STA,139,53
2107,R138 R293 00-03-02,293,1546304400,34 ST-PENN STA,100,258
2108,R138 R293 00-06-00,293,1546304400,34 ST-PENN STA,222,166
2109,R138 R293 00-02-05,293,1546304400,34 ST-PENN STA,101,99
2110,R138 R293 00-00-04,293,1546304400,34 ST-PENN STA,227,107


In [40]:
station_remote_info['remote_unit_id'] = station_remote_info['remote_unit_id'].str.replace('R', '')
station_remote_info['remote_unit_id'] = station_remote_info['remote_unit_id'].astype(int)

In [41]:
station_remote_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   station_name    143 non-null    object 
 1   lat             143 non-null    float64
 2   lon             143 non-null    float64
 3   remote_unit_id  143 non-null    int64  
 4   short_name      143 non-null    object 
dtypes: float64(2), int64(1), object(2)
memory usage: 5.7+ KB


In [42]:
station_remote_info.head()

,station_name,lat,lon,remote_unit_id,short_name
0,Lexington Av/59 St,40.762660,-73.967258,16,LEXINGTON-53 ST
1,Lexington Av/59 St,40.762660,-73.967258,17,LEXINGTON-53 ST
2,Lexington Av/59 St,40.762660,-73.967258,50,59 ST
3,Lexington Av/59 St,40.762660,-73.967258,51,59 ST
4,5 Av/59 St,40.764811,-73.973347,79,5 AVE-59 ST


### Result: 
- The turnstile dataset has :
- A unique id for each row, an id for each turnstile and unit.
- A timestamp, station name, number of entries and exits, the difference between these

## Export the 2 datasets to csv files:

In [43]:
#exporting the data to a csv file
station_remote_info.to_csv('cleaned_station_data.csv', index=False)
df_turn.to_csv('cleaned_turnstile_data.csv', index=False)